# This notebook does a few things:
    
- imports data from the census
    - for demographic data (note that rn the API call doesn't do much, but it is very simple to change) The packages `census` and `states` format our web requests which makes the process very easy for us. Small issue that I haven't been able to track down a list of codes for our info, so pop. data is non-existant. this should not be a hard fix
    - for shape files this is the is the `tract_shapes` dataframe which is very useful. it will likely be the most granular location data we use. also, `school_shapes` dataframe, we will use it to get some info about the current system, and then it will be very useless.
- imports current school districts and concatanates them with the tracts. this combo is not that useful, but i wanted to see how much they lined up. the answer is basically not at all. I also made the `dist_tracts` dataframe which takes all the individual areas of intersection. I think we should stick to either tracts or block groups because this wierd combo will make getting info very hard.
- imports data from NEICES (this is likely not the actual abreviation) which gives us location and population of every single school
- in the next 2h I will hopefull aggregate schools with tracts and this will all become much more functional. 
# Before you run:
- make sure you have all the dependendencies i refer to (including `descartes` which is not listed but is an implicit geopandas dependency)
- Places which say `PATH` require: get the path of this file on your own computer and put it in place of that string (this won't run until you replace all of these, there are 3)

In [12]:
%config InlineBackend.figure_format = 'retina'
from census import Census
from durbango import tqdm
from us import states
import networkx as nx
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [4]:
c = Census("3adb5b5d1ccdb4efb28600c50c1a5d2e2b0c720f")
check = c.sf1.state('NAME', states.PA.fips)
geodf = pd.DataFrame.from_dict(c.sf1.get("NAME", geo={'for': 'tract:*',
                       'in': 'state:{}'.format(states.PA.fips)}))

In [5]:
tabs = pd.DataFrame(c.sf1.tables())
pd.DataFrame.to_clipboard(tabs)
tabs

,name,description,variables
0,P38B,FAMILY TYPE BY PRESENCE AND AGE OF OWN CHILDRE...,https://api.census.gov/data/2010/dec/sf1/group...
1,P38A,FAMILY TYPE BY PRESENCE AND AGE OF OWN CHILDRE...,https://api.census.gov/data/2010/dec/sf1/group...
2,P38D,FAMILY TYPE BY PRESENCE AND AGE OF OWN CHILDRE...,https://api.census.gov/data/2010/dec/sf1/group...
3,P38C,FAMILY TYPE BY PRESENCE AND AGE OF OWN CHILDRE...,https://api.census.gov/data/2010/dec/sf1/group...
4,P38I,FAMILY TYPE BY PRESENCE AND AGE OF OWN CHILDRE...,https://api.census.gov/data/2010/dec/sf1/group...
...,...,...,...
328,PCT22B,GROUP QUARTERS POPULATION BY SEX BY GROUP QUA...,https://api.census.gov/data/2010/dec/sf1/group...
329,PCT22G,GROUP QUARTERS POPULATION BY SEX BY GROUP QUA...,https://api.census.gov/data/2010/dec/sf1/group...
330,PCT22H,GROUP QUARTERS POPULATION BY SEX BY GROUP QUA...,https://api.census.gov/data/2010/dec/sf1/group...
331,PCT22E,GROUP QUARTERS POPULATION BY SEX BY GROUP QUA...,https://api.census.gov/data/2010/dec/sf1/group...


In [6]:
geodf

,NAME,state,county,tract
0,"Census Tract 133.02, Berks County, Pennsylvania",42,011,013302
1,"Census Tract 134.01, Berks County, Pennsylvania",42,011,013401
2,"Census Tract 134.02, Berks County, Pennsylvania",42,011,013402
3,"Census Tract 137.02, Berks County, Pennsylvania",42,011,013702
4,"Census Tract 135, Berks County, Pennsylvania",42,011,013500
...,...,...,...,...
3213,"Census Tract 316, Philadelphia County, Pennsyl...",42,101,031600
3214,"Census Tract 317, Philadelphia County, Pennsyl...",42,101,031700
3215,"Census Tract 318, Philadelphia County, Pennsyl...",42,101,031800
3216,"Census Tract 319, Philadelphia County, Pennsyl...",42,101,031900


In [9]:
PATH_tracts = '/Users/zoeshleifer/promys2020/lab/pa_groups'
PATH_schools = '/Users/zoeshleifer/promys2020/lab/pa_schools'
tract_shapes = gpd.read_file(PATH_tracts).set_index('GEOID10')
school_shapes = gpd.read_file(PATH_schools).set_index('GEOID10')

In [10]:
tract_shapes

,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
GEOID10,,,,,,,,,,,,
420035003001,42,003,500300,1,Block Group 1,G5030,S,993521,99853,+40.3391287,-079.8669912,"POLYGON ((-79.86995 40.34914, -79.86980 40.348..."
420034994002,42,003,499400,2,Block Group 2,G5030,S,202666,0,+40.3225425,-079.8895430,"POLYGON ((-79.88670 40.32117, -79.88671 40.321..."
420034994001,42,003,499400,1,Block Group 1,G5030,S,1487203,206322,+40.3187828,-079.8813494,"POLYGON ((-79.88564 40.31210, -79.88689 40.312..."
420034993002,42,003,499300,2,Block Group 2,G5030,S,1483269,0,+40.3311097,-079.8830403,"POLYGON ((-79.88271 40.33222, -79.88121 40.331..."
420034980002,42,003,498000,2,Block Group 2,G5030,S,2459912,0,+40.3245632,-079.8638781,"POLYGON ((-79.86238 40.31546, -79.86247 40.315..."
...,...,...,...,...,...,...,...,...,...,...,...,...
421090701004,42,109,070100,4,Block Group 4,G5030,S,2351861,95850,+40.8238700,-076.8444760,"POLYGON ((-76.85444 40.81306, -76.85452 40.813..."
421090701006,42,109,070100,6,Block Group 6,G5030,S,1085540,3416,+40.8446326,-076.8224950,"POLYGON ((-76.83057 40.84448, -76.83053 40.844..."
421090707011,42,109,070701,1,Block Group 1,G5030,S,868343,139,+40.8052727,-076.8705870,"POLYGON ((-76.86729 40.81150, -76.86714 40.811..."


In [13]:
borders = {}
for tract in tqdm(tract_shapes.index):
    ser = tract_shapes['geometry'].touches(tract_shapes['geometry'].loc[tract])
    borders[tract] = list(ser.loc[ser].index)

100%|██████████| 9740/9740 [07:48<00:00, 20.77it/s]


this is a giagantic graph of all of PA. It is basically too big to interact with...

In [14]:
graph = nx.Graph(bord_dict)

NameError: name 'bord_dict' is not defined

In [ ]:
bord_dict = borders

In [ ]:
starter = '420035003001'
def get_subgraph(starter='420035003001',max_size=100,levels=20):
    nodes = []
    new_nodes = [starter]
    lev = 0
    for i in tqdm(range(levels)):
        if len(node)> max_size:
            break
        nodes += new_nodes
        nodey = []
        for node in new_nodes:
            nodey += bord_dict[node]
        new_nodes = nodey
        
    return nodes
small_nodes = get_subgraph()

In [ ]:
small_graph = graph.subgraph(small_nodes)
nx.draw(small_graph)

In [ ]:
nx.draw(nx.check_planarity(small_graph,counterexample=True)[1])

In [ ]:
nonplanar = graph.subgraph(['420035523003', '420035520001', '420035524003', '420035513001', '420035520003', '420035030023', '420035520004', '420035520002', '420035513002'])
nx.draw_networkx_labels(nonplanar)

In [ ]:
nx.nodes(nx.check_planarity(small_graph,counterexample=True)[1])

In [ ]:
bord_ser = pd.Series(bord_dict)
tract_shapes['borders'] = bord_ser

In [ ]:
tract_shapes

The next cell takes forever, sorrrry...

In [ ]:
PATH_hs = '/Users/zoeshleifer/promys2020/lab/Public_Schools'
hs_shapes = gpd.read_file(PATH_hs)

In [ ]:
hsg = hs_shapes.groupby('STATE')
hs = hsg.get_group('PA').set_index('NCESID')


In [ ]:
fig = plt.figure(figsize = [20,5])
ax = fig.add_subplot(1, 1, 1)
#school_shapes.plot(ax=ax,linewidth=.4,edgecolor='blue', color='none')
tract_shapes.plot(ax =ax,linewidth=.1,edgecolor='red', color='none')
end_12.plot(ax=ax, markersize=.5)
fig.savefig('/Users/zoeshleifer/promys2020/lab/')

In [ ]:
end_12['ST_GRADE'].value_counts().sum()-490

In [ ]:
end_groups = hs.groupby('END_GRADE')

In [ ]:
end_12 = end_groups.get_group('12')

The calculation in the next cell takes a while, so you only want to do it once. For this reason, the cell underneath copies into a dataframe called `dist_tracts`. I will mess around with that so we are not mutating the original which is annoying to reproduce

In [ ]:
res_intersection = gpd.overlay(school_shapes, tract_shapes, how='intersection')
res_intersection.plot()

In [ ]:
dist_tracts = res_intersection
dist_tracts

In [ ]:
tract_shapes['geometry'].touches(tract_shapes['geometry'].loc['261500'])

In [ ]:
testers = tract_shapes.loc[['420035523003', '420035520001', '420035524003', '420035513001', '420035520003', '420035030023', '420035520004', '420035520002', '420035513002']]
testers.plot(color='none',edgecolor='blue')